In [1]:
import datetime
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

In [2]:
# systems
import os, sys

from dateutil import parser

# scraping websites
import json
import requests
from bs4 import BeautifulSoup

# language processing
from sutime import SUTime

#
sys.path.insert(0, '../src/')
from listing import *

In [3]:
list_id = "mitml"
base_url = "http://mailman.mit.edu/mailman/private/"

local_dir = '../listings/'
local_list_dir = os.path.join(local_dir, list_id)
local_list_dir

# Import URLs and indices
with open(os.path.join(local_list_dir, 'urls.txt'), 'r') as f: 
    urls = [line.rstrip() for line in f.readlines()]
    indices = [os.path.basename(url) for url in urls]

# Google Auth instructions
   
Follow [here](https://developers.google.com/calendar/quickstart/python). But save client secrets as `client_secrets.json` and leave `credentials.json` empty. The following code may have to be run via CMD python shell during initialization. 

In [4]:
# Setup the Gmail API
SCOPES =  'https://www.googleapis.com/auth/calendar'

store = file.Storage('../credentials/credentials.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('../credentials/client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store)

In [5]:
service = build('calendar', 'v3', http=creds.authorize(Http()))

In [10]:
calendarList_df = pd.DataFrame(service.calendarList().list().execute().get('items')).set_index('summary')
calendarList_df.head()

,accessRole,backgroundColor,colorId,conferenceProperties,defaultReminders,description,etag,foregroundColor,id,kind,location,notificationSettings,primary,selected,timeZone
summary,,,,,,,,,,,,,,,
Fraenkel Lab,owner,#92e1c0,13,{'allowedConferenceSolutionTypes': ['eventHang...,"[{'method': 'popup', 'minutes': 30}]",NaN,"""1535508322292000""",#000000,fraenkelabcalendar@gmail.com,calendar#calendarListEntry,Boston,NaN,NaN,NaN,America/New_York
csbphd@mit.edu,reader,#9a9cff,17,{'allowedConferenceSolutionTypes': ['eventHang...,"[{'method': 'popup', 'minutes': 30}]",NaN,"""1535508322292000""",#000000,csbphd@mit.edu,calendar#calendarListEntry,"Cambridge , MA",NaN,NaN,NaN,America/New_York
MIT_talks,owner,#a47ae2,24,{'allowedConferenceSolutionTypes': ['eventHang...,[],NaN,"""1535508322292000""",#000000,ueeje4g086ll88q9k6ji8i5gnc@group.calendar.goog...,calendar#calendarListEntry,NaN,NaN,NaN,NaN,America/New_York
NeuroLINCS,reader,#f691b2,22,{'allowedConferenceSolutionTypes': ['eventHang...,"[{'method': 'popup', 'minutes': 10}]",Calendar for meeting dates and important deadl...,"""1535485974748000""",#000000,bn5kva814kfarmiemjfh5hv53o@group.calendar.goog...,calendar#calendarListEntry,NaN,NaN,NaN,NaN,America/Los_Angeles
Josie-Pee and Johnny Ree,owner,#fad165,12,{'allowedConferenceSolutionTypes': ['eventHang...,"[{'method': 'popup', 'minutes': 30}]",NaN,"""1535508322292000""",#000000,ntolpj5tb9fg1sim1eoskhfflk@group.calendar.goog...,calendar#calendarListEntry,NaN,NaN,NaN,NaN,America/New_York


In [16]:
x = service.calendarList().list().execute().get('items')

In [17]:
x

[{'kind': 'calendar#calendarListEntry',
  'etag': '"1535508322292000"',
  'id': 'fraenkelabcalendar@gmail.com',
  'summary': 'Fraenkel Lab',
  'location': 'Boston',
  'timeZone': 'America/New_York',
  'colorId': '13',
  'backgroundColor': '#92e1c0',
  'foregroundColor': '#000000',
  'accessRole': 'owner',
  'defaultReminders': [{'method': 'popup', 'minutes': 30}],
  'conferenceProperties': {'allowedConferenceSolutionTypes': ['eventHangout']}},
 {'kind': 'calendar#calendarListEntry',
  'etag': '"1535508322292000"',
  'id': 'csbphd@mit.edu',
  'summary': 'csbphd@mit.edu',
  'location': 'Cambridge , MA',
  'timeZone': 'America/New_York',
  'colorId': '17',
  'backgroundColor': '#9a9cff',
  'foregroundColor': '#000000',
  'accessRole': 'reader',
  'defaultReminders': [{'method': 'popup', 'minutes': 30}],
  'conferenceProperties': {'allowedConferenceSolutionTypes': ['eventHangout']}},
 {'kind': 'calendar#calendarListEntry',
  'etag': '"1535508322292000"',
  'id': 'ueeje4g086ll88q9k6ji8i5gnc

In [18]:
[y['id'] for y in x if y.get('summary') == 'MIT_talks']

['ueeje4g086ll88q9k6ji8i5gnc@group.calendar.google.com']

In [14]:
calendarList_df.loc['NeuroLINCS', 'id']

summary
NeuroLINCS    bn5kva814kfarmiemjfh5hv53o@group.calendar.goog...
NeuroLINCS    dlng83226mhp04413j4hp19mlo@group.calendar.goog...
Name: id, dtype: object

Generate new calendar

In [8]:
if list_id not in calendarList_df.index: 

    calendar = {
        'summary': 'mitml',
        'timeZone': 'America/New_York'
    }

    created_calendar = service.calendars().insert(body=calendar).execute()

    calendar_id = created_calendar['id']
    
else: 
    
    calendar_id = calendarList_df.loc[list_id, 'id']

print(calendar_id)

q1oe2irjcmts5kuvciplf5qqb4@group.calendar.google.com


In [9]:
for index in indices[:100]: 
    
    l = Listing(list_id, index)
    
    if l.is_talk: 
        
        data = l.get_parsed_metadata()
        
        event = service.events().insert(calendarId=calendar_id, body=data).execute()
        print('Event created: %s' % (event.get('htmlLink')))

Event created: https://www.google.com/calendar/event?eid=MmdqcWgxc2hlN3RscXAwZnJwOGlkbnJkbGMgcTFvZTJpcmpjbXRzNWt1dmNpcGxmNXFxYjRAZw
Event created: https://www.google.com/calendar/event?eid=ZDJpaTJtcXMxcWgxNnVhbDlvcGxzMjk1cDggcTFvZTJpcmpjbXRzNWt1dmNpcGxmNXFxYjRAZw
Event created: https://www.google.com/calendar/event?eid=MGFjZm9zdmhzaGdqbTc3c2FxbHNkN3Y2YmcgcTFvZTJpcmpjbXRzNWt1dmNpcGxmNXFxYjRAZw
Event created: https://www.google.com/calendar/event?eid=dWk4bTgxZmFyZTgzcW1pdGJ2cjRidGFmdDAgcTFvZTJpcmpjbXRzNWt1dmNpcGxmNXFxYjRAZw
Event created: https://www.google.com/calendar/event?eid=cms3N3VjNzFlcm4wdThrNHVrYXJsMHNmdXMgcTFvZTJpcmpjbXRzNWt1dmNpcGxmNXFxYjRAZw
Event created: https://www.google.com/calendar/event?eid=MmQyOW1pYmZoam03bnQ2YjE3cG9jcHVzajQgcTFvZTJpcmpjbXRzNWt1dmNpcGxmNXFxYjRAZw
Event created: https://www.google.com/calendar/event?eid=ODFsNmNqcDZobHQzdXF0MzlmbG90OHJzajggcTFvZTJpcmpjbXRzNWt1dmNpcGxmNXFxYjRAZw
Event created: https://www.google.com/calendar/event?eid=cG9kNGt0ams0bGo0YmQ

AttributeError: 'NoneType' object has no attribute 'text'